In [1]:
import pandas as pd;import numpy as np
# Import the required libraries and dependencies

import os

import requests

import json

import pandas as pd

import sklearn

from dotenv import load_dotenv

import alpaca_trade_api as tradeapi

#from MCForecastTools import MCSimulation

import datetime

%matplotlib inline

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

custom_dataset = pd.read_csv('ratios_vals_money.csv',
                             infer_datetime_format=True,
                             parse_dates=True,
                             index_col=['Ticker','Year']
                             )


fundementals = custom_dataset.drop(columns=['Close'])



In [2]:
 """
# Introduction to Modeling Libraries in Python. Interfacing Between pandas and Model Code.

Feature engineering: Describes any data tranformation or analytics that ectract infromation from a raw dataset.

The data aggregation and GroupBy tools we have explored in this book are used often in feature engineering context.


# Turning df into a numpy array, use the .values property:
"""
    
display(fundementals[:5])

display(list(fundementals.columns))


display(list(fundementals.values))

Shares (Basic)  Shares (Diluted)   Revenue  Cost of Revenue  Gross Profit  Operating Expenses  Selling, General & Administrative  Research & Development  Depreciation & Amortization_x  Operating Income (Loss)  Non-Operating Income (Loss)  Interest Expense, Net  Pretax Income (Loss), Adj.  Abnormal Gains (Losses)  Pretax Income (Loss)  Income Tax (Expense) Benefit, Net  Income (Loss) from Continuing Operations  Net Extraordinary Gains (Losses)  Net Income  Net Income (Common)  Cash, Cash Equivalents & Short Term Investments  Accounts & Notes Receivable  Inventories  Total Current Assets  Property, Plant & Equipment, Net  Long Term Investments & Receivables  Other Long Term Assets  Total Noncurrent Assets  Total Assets  Payables & Accruals  Short Term Debt  Total Current Liabilities  Long Term Debt  Total Noncurrent Liabilities  Total Liabilities  Share Capital & Additional Paid-In Capital  Treasury Stock  Retained Earnings  Total Equity  Total Liabilities & Equity  \
Ticker Year                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
A      2015-01-01        0.172274          0.168954 -0.106620         0.127958     -0.036993           -0.000036                          -0.066345               -0.208035                       0.179789                -0.085018                     0.059474               0.155789                   -0.071408                 0.102475             -0.047871                           0.118054                                 -0.016521                         -0.157372   -0.030874            -0.029651                                         0.155972                    -0.043646     0.006434              0.055896                         -0.207525                            -0.073306               -0.023020                -0.130092     -0.076172            -0.187889        -0.171002                  -0.152952       -0.122725                     -0.125281          -0.147966                                    0.928680       -1.075697           0.209957      0.051456                   -0.076171   
       2016-01-01        0.162817          0.160931 -0.099310         0.127476     -0.015615           -0.013266                          -0.084542               -0.206883                       0.179789                -0.055833                     0.031520               0.149547                   -0.049564                 0.102475             -0.026186                           0.072776                                 -0.006239                         -0.030193   -0.004714            -0.003491                                         0.205335                    -0.037257     0.002733              0.051442                         -0.204437                            -0.065501               -0.001508                -0.114804     -0.066475            -0.192370        -0.171002                  -0.156650       -0.093715                     -0.107180          -0.136744                                    0.944572       -1.128644           0.243927      0.057234                   -0.066475   
       2017-01-01  

['Shares (Basic)',
 'Shares (Diluted)',
 'Revenue',
 'Cost of Revenue',
 'Gross Profit',
 'Operating Expenses',
 'Selling, General & Administrative',
 'Research & Development',
 'Depreciation & Amortization_x',
 'Operating Income (Loss)',
 'Non-Operating Income (Loss)',
 'Interest Expense, Net',
 'Pretax Income (Loss), Adj.',
 'Abnormal Gains (Losses)',
 'Pretax Income (Loss)',
 'Income Tax (Expense) Benefit, Net',
 'Income (Loss) from Continuing Operations',
 'Net Extraordinary Gains (Losses)',
 'Net Income',
 'Net Income (Common)',
 'Cash, Cash Equivalents & Short Term Investments',
 'Accounts & Notes Receivable',
 'Inventories',
 'Total Current Assets',
 'Property, Plant & Equipment, Net',
 'Long Term Investments & Receivables',
 'Other Long Term Assets',
 'Total Noncurrent Assets',
 'Total Assets',
 'Payables & Accruals',
 'Short Term Debt',
 'Total Current Liabilities',
 'Long Term Debt',
 'Total Noncurrent Liabilities',
 'Total Liabilities',
 'Share Capital & Additional Paid-In C

[array([ 1.72274300e-01,  1.68953919e-01, -1.06620494e-01,  1.27957864e-01,
        -3.69925965e-02, -3.59692704e-05, -6.63454383e-02, -2.08035085e-01,
         1.79789209e-01, -8.50175435e-02,  5.94740869e-02,  1.55789403e-01,
        -7.14079931e-02,  1.02474823e-01, -4.78710814e-02,  1.18053686e-01,
        -1.65211852e-02, -1.57372410e-01, -3.08740461e-02, -2.96511314e-02,
         1.55972195e-01, -4.36462583e-02,  6.43411607e-03,  5.58960456e-02,
        -2.07524853e-01, -7.33062499e-02, -2.30204235e-02, -1.30091959e-01,
        -7.61716092e-02, -1.87889432e-01, -1.71001841e-01, -1.52952073e-01,
        -1.22725362e-01, -1.25280667e-01, -1.47966111e-01,  9.28680480e-01,
        -1.07569655e+00,  2.09957267e-01,  5.14556656e-02, -7.61714547e-02,
        -3.38969462e-02, -8.60417443e-02,  4.86734851e-02, -2.66292102e-01,
        -5.98414926e-02, -4.39549277e-02, -1.83544625e-01, -3.24663353e-01,
        -1.21269614e-01,  1.91050218e-01,  1.44319833e-02,  6.90908659e-02,
         7.7

In [3]:
# The .values attribute is intended to be used when your data is homogeneous- for example, all numeric types.
# If you have heterogeneous data, the result will be an anarray pf py objects

df_copy = fundementals.copy()

# for some models, you may only wish to use a subset of the columns. 

strict_data = ['book_value_per_share_ratio', 'divid_yield_ratio',
               'Earnings_per_share_ratio', 'price_earnings_ratio',
               'gross_margin_ratios','operating_margin_ratio',
               'ret_on_asser_ratio','return_onEquity_ratio',
               'ave_tot_assets','av_inven','average_accts_receivable',
               'asset_turnover','inventoty_turnover','days_sales_in_inven',
               'receivables_turnover_ratio','debt_ratio','debt_to_equ',
               'Interest_coverage_ratio','debt_service_cov_ratio','Asets/Liabili',
               'acid_test','cash_ratio','op_cash_flow_ratio']


tech_values = pd.DataFrame(fundementals.loc[:, strict_data].values,
                           index= fundementals.loc[:, strict_data].index,
                           columns = fundementals.loc[:, strict_data].columns


)


tech_values



book_value_per_share_ratio  divid_yield_ratio  Earnings_per_share_ratio  price_earnings_ratio  gross_margin_ratios  operating_margin_ratio  ret_on_asser_ratio  return_onEquity_ratio  ave_tot_assets  av_inven  average_accts_receivable  asset_turnover  inventoty_turnover  days_sales_in_inven  receivables_turnover_ratio  debt_ratio  debt_to_equ  Interest_coverage_ratio  debt_service_cov_ratio  Asets/Liabili  acid_test  cash_ratio  op_cash_flow_ratio
Ticker Year                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
A      2015-01-01                    0.298685           0.545640                 -0.179214           -220.975469             0.305044                0.701062           -2.169556              -2.356779        0.008561  0.011222                 -0.004159       -3.606347           11.401937            32.012103                   74.831496   -2.736367    -2.972502                -0.545721               -0.716723      -0.365448   0.097962   -1.019746            0.555844
       2016-01-01                    0.351527           0.460428                 -0.028951          -1498.033736             0.411717                1.472155           -0.737256              -0.662646        0.008561  0.011222                 -0.004159       -0.550605           11.358959            32.133227                    0.127962   -3.045145    -2.736978                -0.373349               -0.544351      -0.328392   0.068886   -1.310792            0.356421
       2017-01-01                    0.648109           0.333975                  0.574877            103.204712            -1.421445               -1.596080           -0.096386              -0.092658        0.008561  0.011222                 -0.004159       10.570290           11.047363            33.039560                   -5.877438   -1.210514    -1.163701                 0.093116                0.025217      -0.826095   0.284778   -2.295088           -0.127086
       2018-01-01                    0.525092           0.186200                 -0.431414           -155.694957             1.045871                0.860293            0.703468               0.601282        0.008561  0.011222                 -0.004159       -7.864347           10.166298            35.902941                   48.262151   -1.851635    -1.582668                 0.188853                0.017851      -0.540063   0.465705   -1.527332           -0.326840
       2019-01-01                    0.650770           0.083650                  1.749350             42.994261             2.279759                1.564727            2.376829               0.311280        0.008561  0.011222                 -0.004159       29.956715            9.462521            38.573230                    2.948680   -1.702791    -0.223005                 0.209948                0.341382      -0.587271   3.316033   -2.293193           -2.184114
...                                       ...                ...                       ...                   ...                  ...                     ...                 ...                    ...             ...       ...                       ...             ...                 ...                  ...                         ...         ...          ...                      ...                     ...            ...        ...         ...                 ...
ZYXI   2015-01-01                    1.129715          -1.031959                  0.867034              0.223751             1.156544                0.914218            1.027382               1.026691        0.008561  0.011222                 -0.004159      -

In [4]:
"""
# Creating Model Descriptions with Patsy 
"""

import patsy

y, X = patsy.dmatrices('debt_to_equ ~ cash_ratio + receivables_turnover_ratio', tech_values)



In [5]:
# These Patsy DesignMatrix instances are Numpy nd arrays with additianal metadata

display(np.asarray(y))

display(np.asarray(X))

# Intercept term comes from: This a convention for intercept by adding the term + 0 to the model

patsy.dmatrices('debt_to_equ ~ cash_ratio + receivables_turnover_ratio', tech_values)[1]

# The Paasty objects can be passed directly into algos like numpy.linalg.lstsq, preforms ordinary least squares regression

coef, resid, _, _ = np.linalg.lstsq(X, y)

display(coef)

# The model metadata is retained in the design_info attibute, so you cna reaatach the model column names to the fitted 
# coefficients to obtain a series


array([[-2.97250209],
       [-2.7369784 ],
       [-1.16370103],
       ...,
       [ 1.0137128 ],
       [ 1.01365355],
       [ 1.01748384]])

array([[  1.        ,  -1.01974554,  74.83149623],
       [  1.        ,  -1.31079213,   0.1279619 ],
       [  1.        ,  -2.29508816,  -5.87743793],
       ...,
       [  1.        ,   0.70208729, -23.99782203],
       [  1.        ,   0.70007998, -23.91520229],
       [  1.        ,   0.69701261, -24.05460036]])

C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  del sys.path[0]


array([[-2.49113362e+01],
       [-1.91829914e-03],
       [-6.24555385e-03]])

In [6]:
'book_value_per_share_ratio', 'divid_yield_ratio',
               'Earnings_per_share_ratio', 'price_earnings_ratio',
               'gross_margin_ratios','operating_margin_ratio',
               'ret_on_asser_ratio','return_onEquity_ratio',
               'ave_tot_assets','av_inven','average_accts_receivable',
               'asset_turnover','inventoty_turnover','days_sales_in_inven',
               'receivables_turnover_ratio','debt_ratio','debt_to_equ',
               'Interest_coverage_ratio','debt_service_cov_ratio','Asets/Liabili',
               'acid_test','cash_ratio','op_cash_flow_ratio']

IndentationError: unexpected indent (Temp/ipykernel_47160/2901142083.py, line 2)

In [ ]:
"""
# Data Trasformation in Pasty Formulas 

You can mix py code into your pasty formulas;
when evalutaing the formulas the library will try to find the functions you use in the enclosing scope:
"""


y, X = patsy.dmatrices('y ~ return_onEquity_ratio + np.log(np.abs(book_value_per_share_ratio) + 1)', tech_values)

display(X)

In [ ]:
# Some commonly used variable tranformation incluse standardizing (to mean 0 and variance 1) and centering  (subtracting the mean)

# Pasty has built-in functions for thid purpose:
# See pasty documentaion 

y, Xs = patsy.dmatrices('y ~ standardize(return_onEquity_ratio) + center(book_value_per_share_ratio)', tech_values)


display(Xs)

In [ ]:
"""
Introducntion to statsmodels: a Py library for fitting many kinds of statistical models, performiung statistical test,
and data exploration and visualization. statsmodels contain more 'classical' frequentist methods,
while Bayesian methods and ml models are found other libraries.

# Estimating Linear Models

Linear models in statsmodels have two different main interfaces: array-based and foremula-based.
These are accesses through sthese API modyle import
"""

import statsmodels.api as sm

import statsmodels.formula.api as smf 

# To show how to use thse, we generate a linear model from some random data:

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

# For reproducibility 

np.random.seed(12345)

N = 100

X = np.c_[dnorm(0, 0.4, size=N),
         dnorm(0, 0.6, size=N),
         dnorm(0 , 0.2, size=N)]

eps = dnorm(0, 0.1, size=N)

beta = [0.1, 0.3, 0.5]


y = np.dot(X, beta) + eps

# Linear models are generally fitted with an intercept term as we saw before
# The sm.add_constant function can add an intercept column to an existing matrix:

X_model = sm.add_constant(X)

X_model[:5]

# The sm.OLS class can fir an ordinary least squares linear regression:

model = sm.OLS(y, X)


# The fit method returns a regression result object containin estimates model parameters & disgnostics 

results = model.fit()

display(results.params)


# The summary method on results cna print a model detailing diagnostic output of the model

print(results.summary())

In [ ]:
# The parameter names here have been given the generic names x1, x2 and so on.
# Suppose instead that all of the model parameters are in a df 

data29 = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

data29['y'] = y

display(data29[:5])

# Now we can use the statsmodels formula API and Pasty formula strings:

results = smf.ols('y ~ col0 + col1 + col2', data=data29).fit()

display(results.params)

display(results.tvalues)

# Estimating Time Series Processes 
# Another class of models in statsmodels are for time series analysis. Among the are autoregressive processes,
# Kalman filtering and other and other state space models, multivariate autoregressive models

# Simulate some timeseries data with an autoregressive structue and noise

init_x = 4


import random 
values = [init_x, init_x]

N = 1000

b0 = 0.8

b1 = -0.4

noise = dnorm(0, 0.1, N)
for i in range(N):
    new_cx = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_cx)
    
    
# This data has an AR(2) structure (two lags) with parameters 0.8 and -0.4. WHen you fit an AR model,
# you may not know the numner of lagged tersm to include
# so you can fit the model with some larger number of lags:

MAXLAGS = 5

model = sm.tsa.AR(values)


resultz = model.fit(MAXLAGS)
    
    
# The estimated parameters in the results have intercept first and the estimates for the dirst teo lags next

display(resultz.params)

# statsmodels documentation

In [73]:
"""
# Introduction demo to scikit-learn



"""

from sklearn.preprocessing import OneHotEncoder


sk_data = pd.read_csv('tested.csv')

In [74]:
sk_data.drop(columns=(['Embarked', 'Ticket', 'Cabin', 'Name']), inplace=True)

In [75]:
sk_data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,0,3,male,34.50,0,0,7.8292
1,893,1,3,female,47.00,1,0,7.0000
2,894,0,2,male,62.00,0,0,9.6875
3,895,0,3,male,27.00,0,0,8.6625
4,896,1,3,female,22.00,1,1,12.2875
5,897,0,3,male,14.00,0,0,9.2250
6,898,1,3,female,30.00,0,0,7.6292
7,899,0,2,male,26.00,1,1,29.0000
8,900,1,3,female,18.00,0,0,7.2292
9,901,0,3,male,21.00,2,0,24.1500


In [76]:
y = sk_data['Survived']

X = sk_data.drop(columns='Survived')

# Now we need to specify our models .
X['IsFemale'] = (X['Sex'] == 'female').astype(int)

# then we decide on variables and create numpy arrays:

predictiors = X[['Pclass', 'IsFemale', 'Age']]

X.drop(columns=('Sex'), inplace=True)

X_val = X.fillna(0)

y_val = y.fillna(0)

In [87]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_val.values, y_val.values, random_state=42)

In [95]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=250)

# Similar to statsmodels, we can fit this model to the training data using the models fit method:

model.fit(X_train, y_train)

# Now, we can form predictions for the test dataset using model .predict

y_predict = model.predict(X_test)


C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [103]:
from sklearn.linear_model import LogisticRegressionCV


model_cv = LogisticRegressionCV(10, max_iter=1000)

model_cv.fit(X_train , y_train)

model_cv.predict_proba(X_test)

C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass Cs=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


array([[0.94752227, 0.05247773],
       [0.07639047, 0.92360953],
       [0.94952277, 0.05047723],
       [0.93837236, 0.06162764],
       [0.12218059, 0.87781941],
       [0.92465061, 0.07534939],
       [0.12736436, 0.87263564],
       [0.92287155, 0.07712845],
       [0.12705639, 0.87294361],
       [0.95010521, 0.04989479],
       [0.92757273, 0.07242727],
       [0.11368152, 0.88631848],
       [0.11891623, 0.88108377],
       [0.10216336, 0.89783664],
       [0.95001328, 0.04998672],
       [0.92954883, 0.07045117],
       [0.15384857, 0.84615143],
       [0.93172737, 0.06827263],
       [0.91842508, 0.08157492],
       [0.94175273, 0.05824727],
       [0.94383268, 0.05616732],
       [0.92205045, 0.07794955],
       [0.13915915, 0.86084085],
       [0.93591765, 0.06408235],
       [0.1254302 , 0.8745698 ],
       [0.13023282, 0.86976718],
       [0.10698549, 0.89301451],
       [0.92493307, 0.07506693],
       [0.94356367, 0.05643633],
       [0.93185118, 0.06814882],
       [0.

In [106]:
from sklearn.model_selection import cross_val_score

lst_model = LogisticRegression(C=10)

scores = cross_val_score(lst_model, X_train, y_train, cv=4, n_jobs=-1)

scores

array([1., 1., 1., 1.])

In [ ]:
# This completes chapter 13